In [1]:
from pandas_profiling import ProfileReport
import pandas as pd

In [2]:
df = pd.read_csv("../data/processed/EPC_elec_consump_fuel_poverty.csv")

In [3]:
profile = ProfileReport(df, title="Pandas Profiling Report", explorative=True)
profile.to_file("output.html")

Summarize dataset:  94%|█████████▍| 115/122 [00:06<00:00, 22.98it/s, Calculate cramers correlation]                 c:\Users\lilia\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas_profiling\model\correlations.py:55: UserWarning: There was an attempt to calculate the cramers correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"cramers": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/pandas-profiling/pandas-profiling/issues
(include the error message: 'No data; `observed` has size 0.')
  warnings.warn(
Summarize dataset:  99%|█████████▉| 1414/1422 [04:23<00:01,  5.75it/s, Missing diagram bar]                                                c:\Users\lilia\AppData\Local\Programs\Python\Python39\lib\site-packages\missingno\missingno.py:250: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax1.set_xticklabels(ax1.get_x